In [ ]:
# trazendo a base que foi respondido de ura
import pymysql
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings('ignore')
# Estabelece a conexão com o banco de dados
connection = pymysql.connect(
    host='10.0.6.2',
    user='planejamento',
    password='@Planejamento!123',
    database='atn'
)


query = """
SELECT
  t.Chave3 AS 'CPF',
  t.Chave2 AS 'NOME',
  t.CallID,
  CONCAT(t.ddd, t.fone) AS Telefone,
  t.grupoprincipal,
  t.ResultadoClassificacao,
  t1.codtabulacao,
  t2.descricao,
  f.arc_desc,
  (CASE
    WHEN LOWER(f.arc_desc) LIKE '%movel%' THEN 'MOVEL'
    WHEN LOWER(f.arc_desc) LIKE '%net%' THEN 'NET'
    WHEN LOWER(f.arc_desc) LIKE '%tv%' THEN 'TV'
    WHEN LOWER(f.arc_desc) LIKE '%fixo%' THEN 'FIXO'
    ELSE 'OUTRO'
  END) AS FILA
FROM totalinfo_2024_07 t
LEFT JOIN tabulacaooper t1
  ON t.CallID = t1.callid
LEFT JOIN tabulacao t2
  ON t1.codtabulacao = t2.codtabulacao AND t1.codgrupo = t2.id_grupo
LEFT JOIN arquivo_mailing f
  ON t.id_arquivo = f.ID_ARQUIVOMAILING
WHERE t.Instante >= '2024-07-26 08:00:00'
  AND t.Instante <= '2024-07-27 22:00:00'
  AND t.grupoprincipal IN (6540)
  AND t.ResultadoClassificacao = 'VozHumana'
  AND t1.codtabulacao NOT IN (24, 36, 23, 21, 30, 50, 2, 8, 20, 45, 51, 38, 27, 21, 20);
"""

respondidos_agv = pd.read_sql(query, connection)

# ## Dando nome as filas
respondidos_agv[['grupoprincipal','CPF'] ]= respondidos_agv[['grupoprincipal','CPF']].astype(str)  # converte a coluna para o tipo string
respondidos_agv['NOME'] = respondidos_agv['NOME'] +  ' ' + respondidos_agv['CPF']
respondidos_agv = respondidos_agv[['CPF','NOME','Telefone','FILA']]

respondidos_agv = respondidos_agv.drop_duplicates(['CPF','Telefone'])

resp_movel = respondidos_agv.query("FILA == 'MOVEL'")
resp_fixo = respondidos_agv.query("FILA == 'FIXO'")
resp_tv = respondidos_agv.query("FILA == 'TV'")
resp_net = respondidos_agv.query("FILA == 'NET'")

connection.close()

In [ ]:
resp_movel.to_csv(r"R:\TI\TELEFONIA\Mailings Prontos\2024\Julho\29\Jorgean/respondido_movel_2907.csv",index=False,sep=';')
resp_fixo.to_csv(r"R:\TI\TELEFONIA\Mailings Prontos\2024\Julho\29\Jorgean/respondido_fixo_2907.csv",index=False,sep=';')
resp_tv.to_csv(r"R:\TI\TELEFONIA\Mailings Prontos\2024\Julho\29\Jorgean/respondido_tv_2907_.csv",index=False,sep=';')
resp_net.to_csv(r"R:\TI\TELEFONIA\Mailings Prontos\2024\Julho\29\Jorgean/respondido_net_2907_.csv",index=False,sep=';')